In [70]:
import numpy as np
import pandas as pd
df = pd.read_csv("data (1).csv")
pd.options.mode.chained_assignment = None

Импортируем необходимые бибилотеки, а также читаем и выводим исходный дата фрейм

In [71]:
df.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Во всем файле найдены пустые значения только в двух столбцах, которые содержат информацию о количестве отработанных дней а также о ежемесячном доходе, из чего можно сделать вывод о том, что человек сейчас не трудоустроен, т.е. не может получать за это зарплату

In [72]:
data =  {'na_days_employed_percent': df['days_employed'].isnull().sum()/len(df['days_employed'])*100, 'na_total_income_percent':df['total_income'].isnull().sum()/len(df['total_income'])*100}
df1 = pd.Series(data)
df1

na_days_employed_percent    10.099884
na_total_income_percent     10.099884
dtype: float64

рассчет доли количества пропусков от общего количества значений в столбце в процентах. Pandas позволяет нам увидеть готовую гистограмму нашего датафрейма, для этого необходимо надать на иконку графика в окне вывода отработанного блока кода, и мы увидим гистограмму нашего фрейма данных.

In [73]:
df.loc[(df.days_employed < 0), 'days_employed'] = df.days_employed.abs()
df.fillna(df['days_employed'].median(), inplace=True)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Заполнение данных медианным способом является лучшим решением для количественных переменных, так как на медиану не влияет посторонний "шум". К тому же, количество наших данных велико, поэтому медиана будет равняться среднему значению данных.
данный блок кода проверяет каждое значение в столбце days_employed, и если оно отрицательно, то возвращеает это же значение по модулю, а также заполняет пропуски медианным способом Возможно, отрицательность значений вызвана программным сбоем при заполнении значений

In [74]:
df[['total_income']] = df[['total_income']].astype(int)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


данный блок кода изменяет вещественный тип значений в столбце total_income на целочисленный, используя метод astype

In [75]:
df = df.drop_duplicates()
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


используя метод drop_duplicates избавляемся от строк-дубликатов. Вероятно, дубликаты появились при объединении нескольких фреймов данных.

In [76]:
df['education'] = df['education'].str.capitalize()
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,Высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,Среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,Среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,Среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,Среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,Среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,Среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,Среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


В данном блоке кода мы выбираем столбец education и используя метод str.capitalize приводим значения в нем к одному регистру, а именно - первая буква заглавная, остальные строчные. Также в этом же блоке

In [77]:
dfED = pd.DataFrame({'education': df['education'].drop_duplicates(), 'education_id':  df['education_id'].drop_duplicates()})
dfED

,education,education_id
0,Высшее,0
1,Среднее,1
13,Неоконченное высшее,2
31,Начальное,3
2963,Ученая степень,4


In [78]:
dfFAM_STAT = pd.DataFrame({'family_status': df['family_status'].drop_duplicates(), 'family_status_id':  df['family_status_id'].drop_duplicates()})
dfFAM_STAT

,family_status,family_status_id
0,женат / замужем,0
4,гражданский брак,1
18,вдовец / вдова,2
19,в разводе,3
24,Не женат / не замужем,4



В двух предыдущих блоках мы создали два новых датафрейма, в которых хранятся значения уровня образования и его id в первом и статус семьи и его id во втором соответственно

In [79]:
df = df.drop(columns=['education','family_status'])
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем
21521,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость
21523,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля



удаляем столбцы education и family_status

In [81]:
def tot_inc_cat(row):
    if row['total_income'] <= 30000:
        return "E"
    elif row['total_income'] >= 30001 and row['total_income'] <= 50000:
        return "D"
    elif row['total_income'] >= 50001 and row['total_income'] <= 200000:
        return "C"
    elif row['total_income'] >= 200001 and row['total_income'] <= 1000000:
        return "B"
    else:
        return "A"
    
df['total_income_category'] = df.apply(tot_inc_cat, axis=1)
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B
21521,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21522,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C
21523,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B


Разделяем категории заработка

In [82]:
def purp_cat(row):
    if 'авто' in row['purpose']:
        return 'Операции с автомобилем'
    elif 'недвиж' in row['purpose'] or 'жил' in row['purpose']:
        return 'Операции с недвижимостью'
    elif 'свадьб' in row['purpose']:
        return 'Проведение свадьбы'
    else:
        return 'Получение образования'

df['purpose_category'] = df.apply(purp_cat, axis=1)
df

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья,B,Операции с недвижимостью
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,Операции с автомобилем
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья,C,Операции с недвижимостью
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,Получение образования
4,0,340266.072047,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,Проведение свадьбы
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,1,1,F,компаньон,0,224791,операции с жильем,B,Операции с недвижимостью
21521,0,343937.404131,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C,Операции с автомобилем
21522,1,2113.346888,38,1,1,M,сотрудник,1,89672,недвижимость,C,Операции с недвижимостью
21523,3,3112.481705,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B,Операции с автомобилем


Приводим цели получения кредита к общему виду